# Simple neural networks

In [45]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [39]:
n_inputs = 28*28
n_hidden_1 = 100
n_hidden_2 = 100
n_outputs = 10

In [58]:
# X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
X = keras.Input(shape=(n_inputs,), name='inputs')
hidden_1 = layers.Dense(n_hidden_1, name='hidden_1', activation='relu')(X)
hidden_2 = layers.Dense(n_hidden_2, name='hidden_2', activation='relu')(hidden_1)
output = layers.Dense(n_outputs, name='outputs', activation='softmax')(hidden_2)
model = keras.Model(inputs=X, outputs=output, name='mnist-classifier')

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"])

model.summary()

Model: "mnist-classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
hidden_1 (Dense)             (None, 100)               78500     
_________________________________________________________________
hidden_2 (Dense)             (None, 100)               10100     
_________________________________________________________________
outputs (Dense)              (None, 10)                1010      
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [59]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])

In [60]:
history = model.fit(
    x_train, y_train, batch_size=64, epochs=10, validation_split=0.2,
    callbacks=EarlyStopping(monitor='val_loss', patience=3))

Epoch 1/10
750/750 [==============================] - 2s 2ms/step - loss: 6.1902 - accuracy: 0.7830 - val_loss: 0.8276 - val_accuracy: 0.9032
Epoch 2/10
750/750 [==============================] - 1s 2ms/step - loss: 0.6264 - accuracy: 0.9195 - val_loss: 0.4754 - val_accuracy: 0.9282
Epoch 3/10
750/750 [==============================] - 1s 2ms/step - loss: 0.3615 - accuracy: 0.9391 - val_loss: 0.3318 - val_accuracy: 0.9431
Epoch 4/10
750/750 [==============================] - 1s 2ms/step - loss: 0.2655 - accuracy: 0.9490 - val_loss: 0.2896 - val_accuracy: 0.9458
Epoch 5/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1999 - accuracy: 0.9592 - val_loss: 0.2784 - val_accuracy: 0.9543
Epoch 6/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1626 - accuracy: 0.9653 - val_loss: 0.2326 - val_accuracy: 0.9601
Epoch 7/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1347 - accuracy: 0.9698 - val_loss: 0.2534 - val_accuracy: 0.9572
Epoch 

In [61]:
test_scores = model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.3509 - accuracy: 0.9587


In [62]:
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Test loss: 0.3508969843387604
Test accuracy: 0.9587000012397766


In [63]:
predicted = model.predict(x_test)

In [64]:
predicted[0]

array([2.6473835e-30, 2.6346543e-26, 3.2495604e-16, 3.6663517e-17,
       3.7450123e-34, 5.6709904e-36, 0.0000000e+00, 1.0000000e+00,
       0.0000000e+00, 9.2911201e-19], dtype=float32)

In [65]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [66]:
predicted[1]

array([0.0000000e+00, 5.3639613e-30, 1.0000000e+00, 3.9277357e-26,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [57]:
predicted[2]

array([6.637202e-34, 1.000000e+00, 0.000000e+00, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 7.878013e-22, 0.000000e+00,
       0.000000e+00, 0.000000e+00], dtype=float32)

# Character-level recurren sequence-to-sequence model

https://keras.io/examples/nlp/lstm_seq2seq/

In [67]:
import numpy as np

In [68]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip

['  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current',
 '                                 Dload  Upload   Total   Spent    Left  Speed',
 '',
 '  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0',
 '  0 5986k    0   586    0     0    854      0  1:59:37 --:--:--  1:59:37   852',
 ' 58 5986k   58 3488k    0     0  2160k      0  0:00:02  0:00:01  0:00:01 2158k',
 '100 5986k  100 5986k    0     0  3211k      0  0:00:01  0:00:01 --:--:-- 3209k']

In [69]:
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

In [70]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = "fra.txt"

In [75]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    
for line in lines[:min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [76]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [78]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [80]:
encoder_input_data.shape

(10000, 15, 71)

In [85]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
        
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
            
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [86]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder_out, state_h, state_c = keras.layers.LSTM(latent_dim, return_state=True)(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_outputs, _, _ = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)(decoder_inputs, initial_state=encoder_states)
decoder_outputs = keras.layers.Dense(num_decoder_tokens, activation='softmax')(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [87]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, None, 94)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_11[0][0]                   
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  359424      input_12[0][0]                   
                                                                 lstm[0][1]                   

In [88]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
125/125 [==============================] - 27s 191ms/step - loss: 1.5164 - accuracy: 0.6973 - val_loss: 1.0162 - val_accuracy: 0.7147
Epoch 2/100
125/125 [==============================] - 25s 197ms/step - loss: 0.8872 - accuracy: 0.7596 - val_loss: 0.8132 - val_accuracy: 0.7734
Epoch 3/100
125/125 [==============================] - 22s 175ms/step - loss: 0.6946 - accuracy: 0.8061 - val_loss: 0.7109 - val_accuracy: 0.7914
Epoch 4/100
125/125 [==============================] - 25s 198ms/step - loss: 0.6056 - accuracy: 0.8243 - val_loss: 0.6515 - val_accuracy: 0.8076
Epoch 5/100
125/125 [==============================] - 23s 182ms/step - loss: 0.5490 - accuracy: 0.8389 - val_loss: 0.6173 - val_accuracy: 0.8189
Epoch 6/100
125/125 [==============================] - 23s 182ms/step - loss: 0.5119 - accuracy: 0.8506 - val_loss: 0.5854 - val_accuracy: 0.8276
Epoch 7/100
125/125 [==============================] - 23s 182ms/step - loss: 0.4750 - accuracy: 0.8599 - val_loss: 0.5610 -

Epoch 57/100
125/125 [==============================] - 15s 121ms/step - loss: 0.0941 - accuracy: 0.9708 - val_loss: 0.5836 - val_accuracy: 0.8723
Epoch 58/100
125/125 [==============================] - 40s 323ms/step - loss: 0.0909 - accuracy: 0.9718 - val_loss: 0.5905 - val_accuracy: 0.8715
Epoch 59/100
125/125 [==============================] - 18s 139ms/step - loss: 0.0897 - accuracy: 0.9719 - val_loss: 0.5933 - val_accuracy: 0.8716
Epoch 60/100
125/125 [==============================] - 42s 338ms/step - loss: 0.0869 - accuracy: 0.9725 - val_loss: 0.5996 - val_accuracy: 0.8714
Epoch 61/100
125/125 [==============================] - 18s 139ms/step - loss: 0.0854 - accuracy: 0.9731 - val_loss: 0.6033 - val_accuracy: 0.8712
Epoch 62/100
125/125 [==============================] - 27s 213ms/step - loss: 0.0835 - accuracy: 0.9735 - val_loss: 0.6043 - val_accuracy: 0.8718
Epoch 63/100
125/125 [==============================] - 33s 268ms/step - loss: 0.0816 - accuracy: 0.9741 - val_loss: 0